**Applied Data Science Capstone**

This notebook has been created in the context of the Applied Data Science Capstone that is part of the "IBM Data Science" professional certificate offered by IBM.

_Week 1 peer-graded assignment_

In [27]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


_Week 3 peer-graded assignment_

Below here, is where I read the Wikipedia page of Toronto neighborhoods and import them in a Pandas dataframe.

In [28]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df.Borough != 'Not assigned']
df.shape

(103, 3)

Below here, is where I get Toronto coordinates. They will be used at the end of the notebook for displaying the map of neighborhoods centered in Toronto.

In [29]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: done

# All requested packages already installed.

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Below here, is where I get geospatial data about Toronto neighborhoods, and add them to the neighborhoods' dataframe.

In [30]:
geo = pd.read_csv('https://cocl.us/Geospatial_data')
df = df.join(geo.set_index('Postal Code'), on='Postal Code')
df

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
9,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
11,M3B,North York,Don Mills,43.745906,-79.352188
12,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


Below here, is where I cut off data not strictly related to the city of Toronto, invoke FourSquare APIs for retrieving POIs in Toronto neighborhoods, cluster neighborhoods of Toronto, and display clusters on a map, exactly the same way it is done for the city of New York in the lab "Segmenting and clustering neighborhoods in New York". Necessary dependencies are installed in first.

In [31]:
# Install/import necessary dependencies
# -------------------------------------
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from sklearn.cluster import KMeans
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [32]:
# Cut off neighborhoods that are not in Toronto
# ---------------------------------------------

neighborhoods = df[['Borough','Neighborhood','Latitude','Longitude']]
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031
5,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,Downtown Toronto,Christie,43.669542,-79.422564
8,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [33]:
# Retrieve POIs through FourSquare API calls
# ------------------------------------------

CLIENT_ID = '-- Put your own FourSquare Client ID here, it is what I had done before running the cell, and then replacing the actual ID with this placeholder --' 
CLIENT_SECRET = '-- Put your own FourSquare Client Secret here, it is what I had done before running the cell, and replacing the actual ID with this placeholder --' 
VERSION = '20180604'
LIMIT = 100 
radius = 500 

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        print("---------------")
        print("For this place:")
        print(name)
            
        # create the API request URL
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)        
        
        print("I made this invocation:")
        print(url)
             
        # make the GET request
                
        apiResp = requests.get(url).json()
        
        print("and I obtained this response:")   
        print(apiResp)
        
        # extract items from full response
        
        results = apiResp["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],latitudes=toronto_data['Latitude'],longitudes=toronto_data['Longitude'])

---------------
For this place:
Regent Park, Harbourfront
I made this invocation:
https://api.foursquare.com/v2/venues/explore?&client_id=LH053BNQIPGKBI0WU0EP5UZ124HLZXVX4Z55O1HOYNB5WB2L&client_secret=AW0NG4SS111TVCBGHXFXUTIBFF1OG3HEHJRCUJSST32NA2DH&v=20180604&ll=43.6542599,-79.3606359&radius=500&limit=100
and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a7850ed92b670493e567'}, 'response': {'headerLocation': 'Corktown', 'headerFullLocation': 'Corktown, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 46, 'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486}, 'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '54ea41ad498e9a11e9e13308', 'name': 'Roselle Desserts', 'location': {'address': '362 Ki

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a9175c6a3e18e5e8d6a2'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Bay Street Corridor', 'headerFullLocation': 'Bay Street Corridor, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 32, 'suggestedBounds': {'ne': {'lat': 43.6668015045, 'lng': -79.38328496866619}, 'sw': {'lat': 43.657801495499996, 'lng': -79.3957026313338}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b9d206bf964a520e69136e3', 'name': "Queen's Park", 'location': {'address': 'University Ave.', 'crossStreet': 'at Wellesley Ave.', 'lat': 43.66394609897775, 'lng': -79.39217952520835, 'labeledLatLngs': [{'label': 'display', 'lat': 43.66394609897775, 'lng': -79.392179

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a6dfe5fd9920d4b1c84a'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Downtown Toronto', 'headerFullLocation': 'Downtown Toronto, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 130, 'suggestedBounds': {'ne': {'lat': 43.6616618045, 'lng': -79.37272880013877}, 'sw': {'lat': 43.652661795499995, 'lng': -79.3851453998612}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '57eda381498ebe0e6ef40972', 'name': 'UNIQLO ユニクロ', 'location': {'address': '220 Yonge St', 'crossStreet': 'at Dundas St W', 'lat': 43.65591027779457, 'lng': -79.38064099181345, 'labeledLatLngs': [{'label': 'display', 'lat': 43.65591027779457, 'lng': -79.38064099181345}], 

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a7e57ab1524df82b8341'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'St. Lawrence', 'headerFullLocation': 'St. Lawrence, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 78, 'suggestedBounds': {'ne': {'lat': 43.6559939045, 'lng': -79.36921018606671}, 'sw': {'lat': 43.646993895499996, 'lng': -79.3816256139333}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '574ad72238fa943556d93b8e', 'name': 'Gyu-Kaku Japanese BBQ', 'location': {'address': '81 Church St', 'crossStreet': 'at Adelaide St E', 'lat': 43.651422275497914, 'lng': -79.37504693687086, 'labeledLatLngs': [{'label': 'display', 'lat': 43.651422275497914, 'lng': -79.3750469368708

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a7d1565b2b260536f26e'}, 'response': {'headerLocation': 'The Beaches', 'headerFullLocation': 'The Beaches, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 4, 'suggestedBounds': {'ne': {'lat': 43.680857404499996, 'lng': -79.28682091449052}, 'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4bd461bc77b29c74a07d9282', 'name': 'Glen Manor Ravine', 'location': {'address': 'Glen Manor', 'crossStreet': 'Queen St.', 'lat': 43.67682094413784, 'lng': -79.29394208780985, 'labeledLatLngs': [{'label': 'display', 'lat': 43.67682094413784, 'lng': -79.29394208780985}], 'distance': 89, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Glen

...

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a88608823562b213f852'}, 'response': {'headerLocation': 'Rosedale', 'headerFullLocation': 'Rosedale, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 4, 'suggestedBounds': {'ne': {'lat': 43.6840626045, 'lng': -79.37131878274371}, 'sw': {'lat': 43.675062595499995, 'lng': -79.38374001725632}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4aff2d47f964a520743522e3', 'name': 'Rosedale Park', 'location': {'address': '38 Scholfield Ave.', 'crossStreet': 'at Edgar Ave.', 'lat': 43.68232820227814, 'lng': -79.37893434347683, 'labeledLatLngs': [{'label': 'display', 'lat': 43.68232820227814, 'lng': -79.37893434347683}], 'distance': 327, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['38 Sc

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a810668a7c209cf450c1'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Cabbagetown', 'headerFullLocation': 'Cabbagetown, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 48, 'suggestedBounds': {'ne': {'lat': 43.6724670045, 'lng': -79.3614658826597}, 'sw': {'lat': 43.663466995499995, 'lng': -79.3738847173403}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4b646a6ff964a5205cb12ae3', 'name': 'Cranberries', 'location': {'address': '601 Parliament St.', 'crossStreet': 'at Wellesley St. E', 'lat': 43.6678427705951, 'lng': -79.36940687874281, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6678427705951, 'lng': -79.36940687874281}], 'dis

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a86a4d8e2536eb72c8bf'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'Financial District', 'headerFullLocation': 'Financial District, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 164, 'suggestedBounds': {'ne': {'lat': 43.652929204500005, 'lng': -79.37607280281344}, 'sw': {'lat': 43.6439291955, 'lng': -79.38848759718655}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4ea43642be7ba4918efc5afa', 'name': 'Adelaide Club Toronto', 'location': {'address': '1 First Canadian Place', 'lat': 43.64927944291712, 'lng': -79.38192123284551, 'labeledLatLngs': [{'label': 'display', 'lat': 43.64927944291712, 'lng': -79.38192123284551}], 'distanc

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a62b8e17c5650177487d'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}]}, 'headerLocation': 'The Village', 'headerFullLocation': 'The Village, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 78, 'suggestedBounds': {'ne': {'lat': 43.6703599045, 'lng': -79.37695070062593}, 'sw': {'lat': 43.6613598955, 'lng': -79.3893690993741}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5bd2379cbcbf7a0039a2d7b9', 'name': 'Storm Crow Manor', 'location': {'address': '580 Church St', 'lat': 43.66683985695023, 'lng': -79.38159278029707, 'labeledLatLngs': [{'label': 'display', 'lat': 43.66683985695023, 'lng': -79.38159278029707}], 'distance': 166, 'postalCode': 'M4Y 2E5', '

and I obtained this response:
{'meta': {'code': 200, 'requestId': '5ee4a93bf89b1820a6ddeb44'}, 'response': {'headerLocation': 'Leslieville', 'headerFullLocation': 'Leslieville, Toronto', 'headerLocationGranularity': 'neighborhood', 'totalResults': 16, 'suggestedBounds': {'ne': {'lat': 43.667243904500005, 'lng': -79.31534912291306}, 'sw': {'lat': 43.6582438955, 'lng': -79.32776687708693}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '58d4650e2f91cb5ccec03447', 'name': 'Rorschach Brewing Co.', 'location': {'address': '1001 Eastern Ave', 'crossStreet': 'Woodfield Rd', 'lat': 43.6634831695922, 'lng': -79.31982368639481, 'labeledLatLngs': [{'label': 'display', 'lat': 43.6634831695922, 'lng': -79.31982368639481}], 'distance': 162, 'postalCode': 'M4L 1A8', 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'C

In [24]:
# Build clusters and display them on map
# --------------------------------------

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


NameError: name 'latitude' is not defined